# Data Prep

In [1]:
from datasets import load_dataset, Dataset
import random

dataset_name = "mlabonne/FineTome-100k"
dataset = load_dataset(dataset_name, split="all").shuffle(seed=42)
messages = dataset.map(
    lambda row: {"role": "user", "content": next((item["value"] for item in row["conversations"] if item["from"] == "human"), None)},
    remove_columns=dataset.column_names
)

messages = random.sample(list(messages), 2)

for message in messages:
    print(message['content'])

Please describe the characteristics of a random forest algorithm.
Write Python code to solve the task:
Let's consider a table consisting of n rows and n columns. The cell located at the intersection of i-th row and j-th column contains number i × j. The rows and columns are numbered starting from 1.

You are given a positive integer x. Your task is to count the number of cells in a table that contain number x.

Input

The single line contains numbers n and x (1 ≤ n ≤ 105, 1 ≤ x ≤ 109) — the size of the table and the number that we are looking for in the table.

Output

Print a single number: the number of times x occurs in the table.

Examples

Input

10 5


Output

2


Input

6 12


Output

4


Input

5 13


Output

0

Note

A table for the second sample test is given below. The occurrences of number 12 are marked bold. 

<image>


# Model Prep

In [2]:

from unsloth import FastLanguageModel
import json, time

orpo_path = "orpo.json"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "EITD/orpo_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = NotImplemented,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

responses = []
for message in messages:
    start = time.time()        
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            message['content'],
            "", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, temperature = 1.5, min_p = 0.1)
    
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    end = time.time()         

    response = response.split("Response")[-1].strip()
    responses.append({'time': end - start, 'response': response})

with open(orpo_path, "w", encoding="utf-8") as f:
    json.dump(responses, f, indent=4)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/conda/envs/lab/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: NotImplemented is not recognized, so we'll default to None
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3 MIG 8g.80gb. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 9.0. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.2 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [3]:

from unsloth import FastLanguageModel
import json, time

llama_path = "llama.json"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = NotImplemented,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model)

responses = []
for message in messages:
    start = time.time()        
    inputs = tokenizer.apply_chat_template(
        [message],
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True, temperature = 1.5, min_p = 0.1)
    
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    end = time.time()         

    response = response.split("assistant")[-1].strip()
    responses.append({'time': end - start, 'response': response})

with open(llama_path, "w", encoding="utf-8") as f:
    json.dump(responses, f, indent=4)

Unsloth: NotImplemented is not recognized, so we'll default to None
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3 MIG 8g.80gb. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 9.0. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


---

# LLM Judge

---

In [8]:
import gc
del model
gc.collect()

21503

In [1]:
from flow_judge import Llamafile, EvalInput, FlowJudge, Vllm
from flow_judge.metrics import RESPONSE_FAITHFULNESS_5POINT

# Initialize the model
judge = Vllm()

# Initialize the judge
faithfulness_judge = FlowJudge(
    metric=RESPONSE_FAITHFULNESS_5POINT,
    model=judge
)

# Create a list of inputs and outputs
inputs_batch = [
    [
        {"query": message["content"]},
        {"context": ""},
    ]
    for message in messages
]

def judge(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    outputs_batch = [{"response": item["response"]} for item in data]

    # Create a list of EvalInput
    eval_inputs_batch = [EvalInput(inputs=inputs, output=output) for inputs, output in zip(inputs_batch, outputs_batch)]

    # Run the batch evaluation
    results = faithfulness_judge.batch_evaluate(eval_inputs_batch, save_results=False)

    for i, result in enumerate(results):
        data[i]['score'] = result.score

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

judge(orpo_path)
judge(llama_path)

/opt/conda/envs/lab/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/flowaicom/Flow-Judge-v0.1-AWQ:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


INFO 12-06 00:58:42 awq_marlin.py:90] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 12-06 00:58:42 config.py:389] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-06 00:58:42 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='flowaicom/Flow-Judge-v0.1-AWQ', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(gu

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 12-06 00:58:45 model_runner.py:1025] Loading model weights took 2.1717 GB
INFO 12-06 00:58:46 gpu_executor.py:122] # GPU blocks: 11582, # CPU blocks: 682


VllmError: An unexpected error occurred while initializing vLLM: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":838, please report a bug to PyTorch. 

In [ ]:
def compute_metrics(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    time_sum = 0
    score_sum = 0
    for item in data:
        time_sum += item['time']
        score_sum += item['score']

    print("Avg Inference Time:", time_sum / len(data))
    print("Avg Score:", score_sum / len(data))

print("Orpo metrics:")
compute_metrics(orpo_path)
print("Llama metrics:")
compute_metrics(llama_path)

Orpo metrics:
Avg Inference Time: 1.6291897296905518
Avg Score: 2.0
Llama metrics:
Avg Inference Time: 1.007995843887329
Avg Score: 1.5
